In [50]:
%matplotlib inline
import matplotlib
import seaborn as sns
import datetime as dt
import pandas as pd
import os 
matplotlib.rcParams['savefig.dpi'] = 144

In [16]:
print(dt.datetime.now())

2018-10-26 17:15:10.046771


In [3]:
!which python

/Users/chadleonard/Springboard/anaconda3/bin/python


# ML: Predicting Star Ratings

Our objective is to predict a new venue's popularity from information available when the venue opens.  We will do this by machine learning from a dataset of venue popularities provided by Yelp.  The dataset contains meta data about the venue (where it is located, the type of food served, etc.).  It also contains a star rating. Note that the venues are not limited to restaurants. This tutorial will walk you through one way to build a machine-learning algorithm.

## Metric

Your model will be assessed based on the root mean squared error of the number of stars you predict.  There is a reference solution (which should not be too hard to beat).  The reference solution has a score of 1. Keeping this in mind...

## A note on scoring
It **is** possible to score >1 on these questions. This indicates that you've beaten our reference model - we compare our model's score on a test set to your score on a test set. See how high you can go!


## Download and parse the incoming data

We start by downloading the dataset from Amazon S3:

In [2]:
#!/Users/chadleonard/.local/bin/aws s3 sync s3://dataincubator-course/mldata/ . \
#      --exclude '*' --include 'yelp_train_academic_dataset_business.json.gz'
!aws s3 sync s3://dataincubator-course/mldata/ . --exclude '*' --include 'yelp_train_academic_dataset_business.json.gz'

/bin/sh: aws: command not found


In [13]:
os.getcwd()

'/Users/chadleonard/DataIncubatorFall2017/notebooks/machine_learning'

The training data are a series of JSON objects, in a gzipped file. Python supports gzipped files natively: [gzip.open](https://docs.python.org/2/library/gzip.html) has the same interface as `open`, but handles `.gz` files automatically.

The built-in json package has a `loads()` function that converts a JSON string into a Python dictionary.  We could call that once for each row of the file. [ujson](http://docs.micropython.org/en/latest/library/ujson.html) has the same interface as the built-in `json` library, but is *substantially* faster (at the cost of non-robust handling of malformed json).  We will use that inside a list comprehension to get a list of dictionaries:

In [67]:
import json

print(dt.datetime.now())
f = '/Users/chadleonard/Springboard/work/springboard/data/yelp_academic_dataset_business.json'

json_list = []
df = pd.DataFrame([json.loads(line) for line in open(f, 'r')])
print(df.shape)

print(dt.datetime.now())

2018-10-26 18:24:45.768581
(188593, 15)
2018-10-26 18:25:16.707357


In [69]:
df.head(1)

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state
0,1314 44 Avenue NE,"{'BikeParking': 'False', 'BusinessAcceptsCredi...",Apn5Q_b6Nz61Tq4XzPdf9A,"Tours, Breweries, Pizza, Restaurants, Food, Ho...",Calgary,"{'Monday': '8:30-17:0', 'Tuesday': '11:0-21:0'...",1,51.091813,-114.031675,Minhas Micro Brewery,,T2E 6L6,24,4.0,AB


In [70]:
star_ratings = [row for row in df['stars']]

sum(star_ratings) / len(star_ratings)

3.6315504817251965

In Scikit Learn, the labels to be predicted, in this case, the stars, are always kept in a separate data structure than the features.  Let's get in this habit now, by creating a separate list of the ratings:

In [4]:
star_ratings = [row['stars'] for row in data]

sum(star_ratings) / len(star_ratings)

3.6729137013021247

### Notes:
1. [Pandas](http://pandas.pydata.org/) is able to read JSON text directly.  Use the `read_json()` function with the `lines=True` keyword argument.  While the rest of this notebook will assume you are using a list of dictionaries, you can complete it with dataframes, if you so desire.  Some of the example code will need to be modified in this case.

2. There are obvious miscodings in the data.  There is no need to try to correct them.

## Building models

For many of the questions below, you will need to build and train an estimator that predicts the star rating given certain features.  This could be a custom estimator that you built from scratch, but in most cases will be a pipeline containing custom or prebuilt transformers and an existing estimator.  We will give you hints of how to proceed, but the only requirement for you is to produce a model that does as well, or better, than the reference models we created.  You are welcome to do this however you like. The details are up to you.

The formats of the input and output to the `fit()` and `predict()` methods are ultimately up to you as well, but we recommend that you deal with lists or arrays, for consistency with the rest of Scikit Learn.  It is also a good idea to take the same type of data for the feature matrix in both `fit()` and `predict()`.  While it is tempting to read the stars from the feature matrix X, you should get in the habit of passing the labels as a separate argument to the `fit()` method.

You may find it useful to serialize the trained models to disk.  This will allow to reload it after restarting the Jupyter notebook, without needing to retrain it.  We recommend using the [`dill` library](https://pypi.python.org/pypi/dill) for this (although the [`joblib` library](http://scikit-learn.org/stable/modules/model_persistence.html) also works).  Use
```python
dill.dump(estimator, open('estimator.dill', 'w'))
```
to serialize the object `estimator` to the file `estimator.dill`.  If you have trouble with this, try setting the `recurse=True` keyword args in the call of `dill.dump()`.  The estimator can be deserialized by calling
```python
estimator = dill.load(open('estimator.dill', 'r'))
```

# Questions

Each of the "model" questions asks you to create a function that models the number of stars venues will receive.  It will be passed a list of dictionaries.  Each of these will have the same format as the JSON objects you've just read in.  Some of the keys (like the stars!) will have been removed.  This function should return a list of numbers of the same length, indicating the predicted star ratings.

This function is passed to the `score()` function, which will receive input from the grader, run your function with that input, report the results back to the grader, and print out the score the grader returned.  Depending on how you constructed your estimator, you may be able to pass the predict method directly to the `score()` function.  If not, you will need to write a small wrapper function to mediate the data types.

## city_avg
The venues belong to different cities.  You can image that the ratings in some cities are probably higher than others.  We wish to build an estimator to make a prediction based on this, but first we need to work out the average rating for each city.  For this problem, create a list of tuples (city name, star rating), one for each city in the dataset.

There are many ways to do this; please feel free to experiment on your own.  If you get stuck, the steps below attempt to guide you through the process.

A simple approach is to go through all of the dictionaries in our array, calculating the sum of the star ratings and the number of venues for each city.  At the end, we can just divide the stars by the count to get the average.

We could create a separate sum and count variable for each city, but that will get tedious quickly.  A better approach to to create a dictionary for each.  The key will be the city name, and the value the running sum or running count.

One slight annoyance of this approach is that we will have to test whether a key exists in the dictionary before adding to the running tally.  The collections module's `defaultdict` class works around this by providing default values for keys that haven't been used.  Thus, if we do

In [ ]:
from collections import defaultdict
star_sum = defaultdict(int)
count = defaultdict(int)

we can increment any key of `stars` or `count` without first worrying whether the key exists.  We need to go through the `data` and `star_ratings` list together, which we can do with the `zip()` function.

In [ ]:
for row, stars in zip(data, star_ratings):
    for a in [row.get('city')]:
        a = '+'+a+'+'
        count[a] += 1
        star_sum[a] += stars


Now we can calculate the average ratings.  Again, a dictionary makes a good container.  (*N.B.* Watch out for Python2's integer division.  Make sure at least one of your values is a float to get floating point division.)

In [ ]:
avg_stars = dict()
for city in star_sum:
    # calculate average star rating and store in avg_stars
    avg_stars[city.replace('+','')] = (star_sum[city] / count[city])

avg_stars
len(avg_stars)

There should be 167 different cities:

In [ ]:
assert len(avg_stars) == 167

We can get that list of tuples from the `.items()` method.

In [ ]:
grader.score('ml__city_avg', lambda: avg_stars.items())

## city_model
Now, let's build a custom estimator that will make a prediction based solely on the city of a venue.  It is tempting to hard-code the answers from the previous section into this model, but we're going to resist and do things properly.

This custom estimator will have a `.fit()` method.  It will receive `data` as its argument `X` and `star_ratings` as `y`, and should repeat the calculation of the previous problem there.  Then the `.predict()` method can look up the average rating for the city of each record it receives.

In [18]:
from sklearn import base
from collections import defaultdict
import numpy as np

class CityEstimator(base.BaseEstimator, base.RegressorMixin):
    
    def __init__(self):
        self.avg_stars = dict()
        self.star_sum = defaultdict(int)
        self.count = defaultdict(int)
        self.med = 0

    
    def fit(self, X, y):
        # Store the average rating per city in self.avg_stars
        for row, stars in zip(X, y):
            self.count[row['city']] += 1
            self.star_sum[row['city']] += stars
        for city in self.count:
            self.avg_stars[city] = (self.star_sum[city] / self.count[city])
        self.med = np.median(self.avg_stars.values())
        return self
    
    def predict(self, X):
        return [self.avg_stars[row['city']] if row['city'] in self.avg_stars else self.med for row in X ]
    

Now we can create an instance of our estimator and train it.

In [19]:
city_est = CityEstimator()
city_est.fit(data, star_ratings)

CityEstimator()

And let's see if it works.

There is a problem, however.  What happens if we're asked to estimate the rating of a venue in a city that's not in our training set?

In [20]:
city_est.predict([{'city': 'Timbuktu'}])

[3.6456043956043955]

Solve this problem before submitting to the grader.

In [ ]:
grader.score('ml__city_model', city_est.predict)

## lat_long_model
You can imagine that a city-based model might not be sufficiently fine-grained. For example, we know that some neighborhoods are trendier than others.  Use the latitude and longitude of a venue as features that help you understand neighborhood dynamics.

Instead of writing a custom estimator, we'll use one of the built-in estimators in Scikit Learn.  Since these estimators won't know what to do with a list of dictionaries, we'll build a `ColumnSelectTransformer` that will return an array containing selected keys of our feature matrix.  While it is tempting to hard-code the latitude and longitude in here, this transformer will be more useful in the future if we write it to work on an arbitrary list of columns.

In [56]:
import numpy as np

class ColumnSelectTransformer(base.BaseEstimator, base.TransformerMixin):
    
    def __init__(self, col_names):
        self.col_names = col_names  # We will need these in transform()
    
    def fit(self, X, y=None):
        # This transformer doesn't need to learn anything about the data,
        # so it can just return self without any further processing
        return self
    
    def transform(self, X):
        # Return an array with the same number of rows as X and one
        # column for each in self.col_names
        result = []
        #print X[:4]
        for l in X:
            result.append([l.get(key) for key in self.col_names])
        return result

Let's test it on a single row, just as a sanity check:

In [ ]:
cst = ColumnSelectTransformer(['latitude', 'longitude'])
assert (cst.fit_transform(data[:1])
        == [[data[0]['latitude'], data[0]['longitude']]])

type(cst.fit_transform(data[:1]))

Now, let's feed the output of the transformer in to a `sklearn.neighbors.KNeighborsRegressor`.  As a sanity check, we'll test it with the first 5 rows.  To truly judge the performance, we'd need to make a test/train split.

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
import numpy as np

n=50
cols = ['stars']
truescore = [l.get(key) for key in cols for l in data[:n]]
data_transform = cst.fit_transform(data)

knn = KNeighborsRegressor(n_neighbors=5, weights='distance', algorithm='auto', \
                    leaf_size=30, p=2,  metric='minkowski', metric_params=None, n_jobs=1)
knn.fit(data_transform, star_ratings)
test_data = data[:n]
test_data_transform = cst.transform(test_data)

print knn.score(test_data_transform,truescore)
knn.get_params()

In [ ]:
knn = KNeighborsRegressor(n_neighbors=5,weights='distance', metric='hamming')
knn.fit(data_transform, star_ratings)
test_data = data[:n]
test_data_transform = cst.transform(test_data)
print knn.score(test_data_transform,truescore)
#knn.get_params()

Instead of doing this by hand, let's make a pipeline.  Remember that a pipeline is made with a list of (name, transformer-or-estimator) tuples. 

In [ ]:
from sklearn.pipeline import Pipeline

cols = ['latitude', 'longitude']

pipe = Pipeline([
   ('cst', ColumnSelectTransformer(cols)) , # ColumnSelectTransformer
   ('knn', KNeighborsRegressor(n_neighbors=5,weights='distance',p=1, metric='minkowski')) # KNeighborsRegressor
    ])

n = len(data)
print n
pipe.fit(data[:20000], star_ratings[:20000])
final = pipe.predict(data[:n])
#print final[:15]
print pipe.score(data[:n], star_ratings[:n])
kdict = pipe.get_params()

#print [(k,kdict[k]) for k in kdict ][0]# if isinstance(kdict[k], str)]
print kdict['knn']
print kdict['cst']
        


This should work the same way.

In [ ]:
pipe.fit(data, star_ratings)
pipe.predict(data)

In [ ]:
pipe.fit(data[:10], star_ratings[:10])
print pipe.predict(data[:10])
print star_ratings[:10]

The `KNeighborsRegressor` takes the `n_neighbors` hyperparameter, which tells it how many nearest neighbors to average together when making a prediction.  There is no reason to believe that 5 is the optimum value.  Determine a better value of this hyperparameter.   There are several ways to do this:

1. Use [train_test_split](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html#sklearn.model_selection.train_test_split) to split your data in to a training set and a test set.  Score the performance on the test set.  After finding the best hyperparameter, retrain the model on the full data at that hyperparameter value.

2. Use [cross_val_score](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html#sklearn.model_selection.cross_val_score) to return cross-validation scores on your data for various values of the hyperparameter.  Choose the best one, and retrain the model on the full data.

3. Use [GridSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV) to do the splitting, training, and grading automatically.  `GridSearchCV` takes an estimator and acts as an estimator.  You can either give it the `KNeighborsRegressor` directly and put it in a pipeline, or you can pass the whole pipeline into the `GridSearchCV`.  In the latter case, remember that the hyperparameter `param` of an estimator named `est` in a pipeline becomes a hyperparameter of the pipeline with name `est__param`.

No matter which you choose, you should consider whether the data need to be shuffled.  The default k-folds split doesn't shuffle.  This is fine, if the data are already random.  The code below will plot a rolling mean of the star ratings.  Do you need to shuffle the data?

In [ ]:
from pandas import Series
import matplotlib.pyplot as plt

plt.plot(Series.rolling(Series(star_ratings), window=1000).mean())

Once you've found a good value of `n_neighbors`, submit the model to the grader.  (*N.B.* "Good" is a relative measure here.  The reference solution has a r-squared value of only 0.02.  There is just rather little signal available for modeling.)

In [5]:
from sklearn import preprocessing, model_selection
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor
import pandas as pd
import datetime as dt

In [ ]:
st = dt.datetime.now()
print st
cols = ['longitude','latitude']

# We are going to first scale and then apply KNeighbors.
# Pipeline allows us to chain together multiple transformers and then a regressor or classifier
pipe = Pipeline([
   ('cst', ColumnSelectTransformer(cols)) , # ColumnSelectTransformer
   ('scaling', preprocessing.StandardScaler(copy=True)),
   ('knn', KNeighborsRegressor(weights='uniform')) # KNeighborsRegressor
    ])

cv = model_selection.ShuffleSplit(n_splits=20, test_size=0.2, random_state=42)
print dt.datetime.now()
# parameters to Pipeline take the form [label]__[estimator_param]
param_grid = { "knn__n_neighbors": range(64, 100, 4) }
lat_long_est = model_selection.GridSearchCV(pipe, 
                                                       param_grid=param_grid, cv=cv, 
                                                       scoring='r2')
print dt.datetime.now()
lat_long_est.fit(data, star_ratings)
print dt.datetime.now()
lat_long_est_results = lat_long_est.cv_results_
lat_long_est_score = pd.DataFrame.from_items(
    [('n_neighbors',[rec['knn__n_neighbors'] for rec in lat_long_est_results['params']]),
     ('score',lat_long_est_results['mean_test_score'])])

plt.plot(lat_long_est_score.n_neighbors, lat_long_est_score.score, label="lat_long_cv_score")
plt.xlabel('n_neighbors')
plt.ylabel('score')
plt.legend(loc='lower center')
plt.show()

st = dt.datetime.now()
print st

In [ ]:
lat_long_est_results['mean_test_score']
lat_long_est_results['params']

In [ ]:
lat_long_est_score

In [ ]:
grader.score('ml__lat_long_model', lat_long_est.predict) 

*Item for thought:* Why do we choose a non-linear model for this estimator?

*Extension:* Use a `sklearn.ensemble.RandomForestRegressor`, which is a more powerful non-linear model.  Can you get better performance with this than with the `KNeighborsRegressor`?

## category_model
While location is important, we could also try seeing how predictive the
venue's category is.  Build an estimator that considers only the categories.

The categories come as a list of strings, but the built-in estimators all need numeric input.  The standard way to deal with categorical features is **one-hot encoding**, also known as dummy variables.  In this approach, each category gets its own column in the feature matrix.  If the row has a given category, that column gets filled with a 1.  Otherwise, it is 0.

The `ColumnSelectTransformer` from the previous question can be used to extract the categories column as a list of strings.  Scikit Learn provides [DictVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.DictVectorizer.html#sklearn.feature_extraction.DictVectorizer), which takes in a list of dictionaries.  It creates a column in the output matrix for each key in the dictionary and fills it with the value associated with it.  Missing keys are filled with zeros.  Therefore, we need only build a transformer that takes a list strings to a dictionary with keys given by those strings and values one.

In [ ]:
class DictEncoder(base.BaseEstimator, base.TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        # X will come in as a list of lists of lists.  Return a list of
        # dictionaries corresponding to those inner lists.
        ls = []
        for li in X:
            ls.append({i:1 for i in li[0]})
        return ls

That should allow this to pass:

In [ ]:
assert (DictEncoder().fit_transform([[['a']], [['b', 'c']]])
        == [{'a': 1}, {'b': 1, 'c': 1}])

Set up a pipeline with your `ColumnSelectTransformer`, your `DictEncoder`, the `DictVectorizer`, and a regularized linear model, like `Ridge`, as the estimator.  This model will have a large number of features, one for each category, so there is a significant danger of overfitting.  Use cross validation to choose the best regularization parameter.

In [6]:
from sklearn.pipeline import Pipeline
from sklearn import linear_model, preprocessing, model_selection
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt

In [ ]:
cst = ColumnSelectTransformer(['categories'])
vect = DictVectorizer(sparse=False)
v = vect.fit_transform(DictEncoder().fit_transform(cst.fit_transform(data)))
av = vect.inverse_transform(v)

In [ ]:
len(vect.vocabulary_)
{k: vect.vocabulary_[k] for k in vect.vocabulary_.keys()[:20]}

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(data, star_ratings, test_size=0.3)

cols = ['categories']
pipe = Pipeline([
   ('cst', ColumnSelectTransformer(cols)) , # ColumnSelectTransformer
    ('denc', DictEncoder()),
    ('vect', DictVectorizer()),
    ('tf_idf', TfidfTransformer() ),
   ('ridge', linear_model.Ridge()) # Ridge Regression
    ])

cv = model_selection.ShuffleSplit(n_splits=20, test_size=0.2, random_state=42)
param_grid = { "ridge__alpha": range(2, 8, 2) }
category_est_cv = model_selection.GridSearchCV(pipe, param_grid=param_grid, cv=cv, 
                                                       scoring='r2')
print dt.datetime.now()
category_est_cv.fit(X_train, y_train)
#category_est_cv.fit(data, star_ratings)
category_est_cv_results = category_est_cv.cv_results_
category_est_cv_score = pd.DataFrame.from_items(
    [('alpha',[rec['ridge__alpha'] for rec in category_est_cv_results['params']]),
     ('score',category_est_cv_results['mean_test_score'])])

plt.plot(category_est_cv_score.alpha, category_est_cv_score.score, label="category_est_cv_score")
plt.xlabel('ridge__alpha')
plt.ylabel('score')
plt.legend(loc='lower center')
plt.show()

print dt.datetime.now()

In [ ]:
category_est_cv_score

In [ ]:
category_est = Pipeline([
    ('cst', ColumnSelectTransformer(cols)) , # ColumnSelectTransformer
    ('denc', DictEncoder()),
    ('vect', DictVectorizer(sparse=False)),
    ('tf_idf', TfidfTransformer() ),
   ('ridge', linear_model.Ridge(alpha=4)) # Ridge Regression
    ])
#category_est.fit(data,star_ratings)
category_est.fit(X_train,y_train)
category_est.score(X_test,y_test)


In [ ]:
grader.score('ml__category_model', category_est.predict)  # Edit to appropriate name
#grader.score('ml__category_model', category_est_cv.predict)  # Edit to appropriate name

*Extension:* Some categories (e.g. Restaurants) are not very specific.  Others (Japanese sushi) are much more so.  One way to deal with this is with an measure call term-frequency-inverse-document-frequency (TF-IDF).  Add in a `sklearn.feature_extraction.text.TfidfTransformer` between the `DictVectorizer` and the linear model, and see if that improves performance.

*Extension:* Can you beat the performance of the linear estimator with a
non-linear model?

## attribute_model
There is even more information in the attributes for each venue.  Let's build an estimator based on these.

Venues attributes may be nested:
```
{
  'Attire': 'casual',
  'Accepts Credit Cards': True,
  'Ambiance': {'casual': False, 'classy': False}
}
```
We wish to encode them with one-hot encoding.  The `DictVectorizer` can do this, but only once we've flattened the dictionary to a single level, like so:
```
{
  'Attire_casual' : 1,
  'Accepts Credit Cards': 1,
  'Ambiance_casual': 0,
  'Ambiance_classy': 0
}
```

Build a custom transformer that flattens the attributes dictionary.  Place this in a pipeline with a `DictVectorizer` and a regressor.

You may find it difficult to find a single regressor that does well enough.  A common solution is to use a linear model to fit the linear part of some data, and use a non-linear model to fit the residual that the linear model can't fit.  Build a residual estimator that takes as an argument two other estimators.  It should use the first to fit the raw data and the second to fit the residuals of the first.

In [7]:
from sklearn import base
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn import preprocessing, model_selection, linear_model
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
#from sklearn.base import BaseEstimator, RegressorMixin, TransformerMixin
from sklearn import ensemble

import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
X = data
y = [row['stars'] for row in data]

In [ ]:
class AttributeSelectTfmRecursive(base.BaseEstimator, base.TransformerMixin):
    
    def __init__(self, col_names='attributes'):
        self.col_names = col_names  # We will need these in transform()
        self.pr_dict = {1:'one', 2:'two', 3:'three', 4:'four'} # a dict for price range lookups
    
    def fit(self, X, y=None):
        # This transformer doesn't need to learn anything about the data,
        # so it can just return self without any further processing
        return self
    
    def parse_dict(self, key, val):
        l = []
        if isinstance(val,dict):
            for k, v in val.items():
                if not isinstance(v,dict):
                    l.append((key + '_'+ k,v))
                else:
                    self.parse_dict(key + '_'+ k, v)
        else:
            l.append((key,self.pr_dict[val]) if key == 'Price Range' else (key.title(),val))         
        return l
    
    def transform(self, X):
        result = []
        for attr in [dx.get(self.col_names) for dx in X]:
            ls = []
            for ke,va in attr.items():
                ls += self.parse_dict(ke,va)
            result.append({x[0]:x[1] for x in ls })
        return result

In [ ]:
class EnsembleTransformer(base.BaseEstimator, base.TransformerMixin):
    
    def __init__(self, base_estimator, residual_estimators):
        self.base_estimator = base_estimator
        self.residual_estimators = residual_estimators
    
    def fit(self, X, y):
        self.base_estimator.fit(X, y)
        y_err = y - self.base_estimator.predict(X)
        for est in self.residual_estimators:
            est.fit(X, y_err)                     # fitting the error (y - y_pred) into KNNreg and then RandForestReg
        return self
    
    def transform(self, X):
        all_ests = [self.base_estimator] + list(self.residual_estimators)
        return np.array([est.predict(X) for est in all_ests]).T

In [135]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(data, star_ratings, test_size=0.2, random_state=0)

In [ ]:
print dt.datetime.now()
r2_list = []
for xnum in range(15,40,5):
    for num in range(3,10,2):
        ensemble_pipe = Pipeline([
               ('ast', AttributeSelectTfmRecursive()),
               ('vect', DictVectorizer(sparse=False)), 
               ('ensemble', EnsembleTransformer(
                        linear_model.LinearRegression(),
                        (KNeighborsRegressor(n_neighbors=num, algorithm='kd_tree'),           
                         ensemble.RandomForestRegressor(min_samples_leaf=xnum)))), 
                ('blend', linear_model.LinearRegression())                       
            ])
        #ensemble_pipe.fit(X_train[:n], y_train[:n]) 
        ensemble_pipe.fit(X_train, y_train) 
        r2_list.append((ensemble_pipe.score(X_test,y_test),( num, xnum)))
print dt.datetime.now()

In [ ]:
print dt.datetime.now()
attribute_est = Pipeline([
   ('ast', AttributeSelectTfmRecursive()),
   ('vect', DictVectorizer(sparse=False)), 
   ('ensemble', EnsembleTransformer(
            linear_model.LinearRegression(),
            (KNeighborsRegressor(n_neighbors=5, algorithm='kd_tree'),           
             ensemble.RandomForestRegressor(min_samples_leaf=20)))), 
    ('blend', linear_model.LinearRegression())                       
])
#ensemble_pipe.fit(X_train[:n], y_train[:n]) 
attribute_est.fit(X_train, y_train) 
print '{},{} ==> {}'.format(5,20,attribute_est.score(X_test,y_test))
print dt.datetime.now()

In [ ]:
# (10,40) ==> 0.0876832516126
print ensemble_pipe.score(X_test,y_test)
sorted(r2_list, reverse=True)

In [ ]:
grader.score('ml__attribute_model', attribute_est.predict)  # Edit to appropriate name
#grader.score('ml__attribute_model', ensemble_pipe.predict)  # Edit to appropriate name

In [139]:
import dill

with open('city_est.dill', 'w') as est:
    dill.dump(city_est, est)
    
with open('lat_long_est.dill', 'w') as est:
    dill.dump(lat_long_est, est)
    
with open('category_est.dill', 'w') as est:
    dill.dump(category_est, est)

with open('attribute_est.dill', 'w') as est:
    dill.dump(attribute_est, est)


In [10]:
import dill

with open('city_est.dill', 'r') as est:
    city_est = dill.load(est)
    
with open('lat_long_est.dill', 'r') as est:
    lat_long_est = dill.load(est)
    
with open('category_est.dill', 'r') as est:
    category_est = dill.load(est)
    
with open('attribute_est.dill', 'r') as est:
    attribute_est = dill.load(est)

## full_model
So far we have only built models based on individual features.  Now we will build an ensemble regressor that averages together the estimates of the four previous regressors.

In order to use the existing models as input to an estimator, we will have to turn them into transformers.  (A pipeline can contain at most a single estimator.)  Build a custom `ModelTransformer` class that takes an estimator as an argument.  When `fit()` is called, the estimator should be fit.  When `transform()` is called, the estimator's `predict()` method should be called, and its results returned.

Note that the output of the `transform()` method should be a 2-D array with a single column, in order for it to work well with the Scikit Learn pipeline.  If you're using Numpy arrays, you can use `.reshape(-1, 1)` to create a column vector.  If you are just using Python lists, you will want a list of lists of single elements.

In [129]:
class EstimatorTransformer(base.BaseEstimator, base.TransformerMixin):
    
    def __init__(self, estimator):
        # What needs to be done here?
        self.estimator = estimator
    
    def fit(self, X, y):
        # Fit the stored estimator.
        # Question: what should be returned?
        # None will be returned without using the 'return' keyword
        self.estimator.fit(X,y)
    
    def transform(self, X):
        # Use predict on the stored estimator as a "transformation".
        # Be sure to return a 2-D array.
        #print type(X)
        return [[x] for x in self.estimator.predict(X)]

This should work as follows:

In [130]:
city_trans = EstimatorTransformer(city_est)
city_trans.fit(data, star_ratings)
assert ([r[0] for r in city_trans.transform(data[:5])]
        == city_est.predict(data[:5]))
city_trans.transform(data[:5])

[[3.670054772869717],
 [3.7447916666666665],
 [3.7447916666666665],
 [3.7447916666666665],
 [3.7447916666666665]]

In [118]:
print dt.datetime.now()
lat_long_trans = EstimatorTransformer(lat_long_est)
lat_long_trans.fit(data, star_ratings)
print dt.datetime.now()

2018-02-09 16:09:07.038317
2018-02-09 16:12:10.011209


In [119]:
print lat_long_trans.transform(data[:10])
print dt.datetime.now()

<type 'list'>
[[ 4.03676471]
 [ 3.58088235]
 [ 3.57352941]
 [ 3.58823529]
 [ 3.57352941]
 [ 3.56617647]
 [ 3.32352941]
 [ 3.33088235]
 [ 3.39705882]
 [ 3.38235294]]
2018-02-09 16:12:57.902881


In [107]:
print dt.datetime.now()
category_trans = EstimatorTransformer(category_est)
category_trans.fit(data, star_ratings)
print dt.datetime.now()

2018-02-09 15:59:21.579211
2018-02-09 15:59:22.551744


In [108]:
print category_trans.transform(data[:10])

<type 'list'>
[[3.2671231067549185], [3.0575147644688747], [3.2245305166167757], [3.2856027840542259], [3.3059986595917499], [3.5152689546274112], [4.0953892314246305], [3.9409850379812381], [4.1046161392687246], [3.3867762665891852]]


In [109]:
print dt.datetime.now()
attribute_trans = EstimatorTransformer(attribute_est)
attribute_trans.fit(data, star_ratings)
print dt.datetime.now()

2018-02-09 16:00:22.220097
2018-02-09 16:04:04.649432


In [122]:
t = attribute_trans.transform(data[:10])
np.array(t).shape
type(data)

<type 'list'>


list

Create an instance of `ModelTransformer` for each of the previous four models. Combine these together in a single feature matrix with a
[FeatureUnion](http://scikit-learn.org/stable/modules/generated/sklearn.pipeline.FeatureUnion.html#sklearn.pipeline.FeatureUnion).

In [ ]:
print lat_long_est.get_params
print city_est.get_params
print category_est.get_params
print attribute_est.get_params

In [131]:
from sklearn.pipeline import FeatureUnion
import datetime as dt
print dt.datetime.now()
union = FeatureUnion([
    # FeatureUnions use the same syntax as Pipelines
    ('city_trans', EstimatorTransformer(city_est)),
    ('lat_long_trans', EstimatorTransformer(lat_long_est)),
    ('category_trans', EstimatorTransformer(category_est)),
    ('attribute_trans', EstimatorTransformer(attribute_est))
    ])
union.fit(data, star_ratings)
print dt.datetime.now()

2018-02-09 16:35:57.144914
2018-02-09 16:42:29.604815


In [99]:
cst = ColumnSelectTransformer(['city','attributes','categories','longitude','latitude'])
#cst.fit_transform(data[:3])

In [121]:
#arr = np.array(cst.fit_transform(data[:10]))
#print arr.shape
#print type(arr)
#union.transform(arr)
union.transform(data[:3])
#print data[:1][0]
#[r[0] for r in union.transform(data[:5])]
#union.get_params

AttributeError: 'list' object has no attribute 'shape'

This should return a feature matrix with four columns.

In [100]:
#union.fit(data, star_ratings)
#trans_data = union.transform(cst.fit_transform(data[:10]))
test_data = np.array(cst.fit_transform(data[:1]))
print test_data.shape
print test_data
#print type(trans_data)
#print trans_data.shape
#assert trans_data.shape == (10, 4)

(1, 5)
[[u'Phoenix' {u'By Appointment Only': True}
  [u'Doctors', u'Health & Medical'] -111.983758 33.499313]]


In [133]:
data[:1] # u'Phoenix De Forest
print

Finally, use a pipeline to combine the feature union with a linear regression (or another model) to weight the predictions.

In [138]:
full_est = Pipeline([
    ("union", union),
    ("linreg", linear_model.LinearRegression(fit_intercept=True))
    ])
full_est.fit(X_train, y_train)

AttributeError: 'list' object has no attribute 'shape'

In [ ]:
grader.score('ml__full_model', full_est.predict)  # Edit to appropriate name

*Extension:* By combining our models with a linear model, we will be unable to notice any correlation between features.  We don't expect all attributes to have the same effect on all venues.  For example, "Ambiance: divey" might be a bad indicator for a restaurant but a good one for a bar.  Nonlinear models can pick up on this interaction.  Replace the linear model combining the predictions with a nonlinear one like [RandomForestRegressor](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn.ensemble.RandomForestRegressor).  Better yet, use the nonlinear model to fit the residuals of the linear model.

The score for this question is just a ratio of the score of your model to the score of a reference solution.  Can you beat the reference solution and get a score greater than 1.0?

*Copyright &copy; 2016 The Data Incubator.  All rights reserved.*